# Web Spider for AFL Data
#### Quick Start

In [22]:
# import libraries
import pandas as pd
import requests
from fake_useragent import UserAgent
from time import sleep


from IPython.core.display_functions import display

from bs4 import BeautifulSoup
import re

import warnings

In [31]:
# Supresses scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# display the full tables
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

debug = True

In [29]:
# Query and Insert the old database !
games = pd.read_csv(r'./data/games.csv')
players = pd.read_csv(r'./data/players.csv')
stats = pd.read_csv(r'./data/stats.csv')

In [19]:
if debug is True:
    display(games.head())

,gameId,year,round,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall
0,2021R101,2021,R1,18-Mar-2021,M.C.G.,7:25 PM,49218,Richmond,105,Carlton,80,0.0
1,2021R102,2021,R1,19-Mar-2021,M.C.G.,7:50 PM,46051,Collingwood,53,Western Bulldogs,69,0.0
2,2021R103,2021,R1,20-Mar-2021,M.C.G.,1:45 PM,21365,Melbourne,80,Fremantle,58,0.0
3,2021R104,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.0
4,2021R105,2021,R1,20-Mar-2021,Docklands,7:25 PM,25128,Essendon,91,Hawthorn,92,0.0


In [20]:
if debug is True:
    display(players.head())

,playerId,displayName,height,weight,dob,position,origin
0,2021661124,"Berry, Sam",181,81,12-Feb-2002,"Midfield, Forward",Gippsland Power
1,2012662083,"Brown, Luke",181,81,22-Sep-1992,Defender,Norwood
2,2020665315,"Butts, Jordon",198,83,31-Dec-1999,Defender,Murray Bushrangers
3,2021678665,"Cook, Brayden",189,80,18-Jul-2002,"Midfield, Forward",South Adelaide
4,2019683521,"Davis, Ben",187,85,19-May-1997,Forward,UNSW


In [32]:
if debug is True:
    display(stats.head())

,gameId,team,year,round,playerId,displayName,gameNumber,Disposals,Kicks,Marks,Handballs,Goals,Behinds,Hit Outs,Tackles,Rebounds,Inside 50s,Clearances,Clangers,Frees,Frees Against,Brownlow Votes,Contested Possessions,Uncontested Possessions,Contested Marks,Marks Inside 50,One Percenters,Bounces,Goal Assists,% Played,Subs
0,2021R104,Adelaide,2021,R1,2021661124,"Berry, Sam",1,8,6,1,2,0,1,0,7,0,2,3,2,2,2,0,4,5,0,0,0,0,1,80,-
1,2021R104,Adelaide,2021,R1,2012662083,"Brown, Luke",168,5,2,0,3,0,0,0,2,0,0,0,0,1,0,0,2,3,0,0,0,0,0,23,Off
2,2021R104,Adelaide,2021,R1,2020665315,"Butts, Jordon",3,10,5,3,5,0,0,0,0,5,0,0,3,1,0,0,5,5,1,0,8,0,0,93,-
3,2021R104,Adelaide,2021,R1,2018689604,"Doedee, Tom",31,13,9,4,4,0,0,0,4,5,1,1,0,2,0,0,8,6,0,0,7,0,0,84,-
4,2021R104,Adelaide,2021,R1,2018703883,"Frampton, Billy",9,14,10,8,4,2,2,7,0,2,0,0,1,0,0,0,5,9,3,4,2,0,0,90,-


In [104]:
def query_player_info(url):

    # Crawling Website
    url_player= url
    player_datas = requests.get(url=url_player, headers=headers)
    html_players = player_datas.text
    # For Loop crawling, time interval is demanded.
    sleep(0.25)
    soup_players = BeautifulSoup(html_players, 'lxml')
    c_play_infos = soup_players.find_all('center')

    temp_player_info = re.match(r'.*?Born:(\d*-\w*-\d*) \(Debut.*?Height:(\d*).*?Weight:(\d*) kg', c_play_infos[0].text, flags=re.S)

    player_dob = temp_player_info.group(1)
    player_height = temp_player_info.group(2)
    player_weight = temp_player_info.group(3)

    return [player_dob, int(player_height), int(player_weight)]


In [133]:
new_players_data = players.copy()
new_stats_data = stats.copy()
new_games_data= games.copy()

In [4]:
# Crawling Website
url='https://afltables.com/afl/seas/2022.html'
# Fake a user header
headers = {'User-Agent': UserAgent().random}


datas = requests.get(url=url, headers=headers)
html = datas.text

# For Loop crawling, time interval is demanded.
sleep(1)

In [5]:
soup = BeautifulSoup(html, 'lxml')


In [88]:
re_date_time_att_venue = re.compile(r'\w* (\d*\-\w*\-\d*) (\d*\:\d*\s\w*) .*?Att\: (\d*\,\d*) \w*\: (.*)')

re_find_player_info = re.compile(r'.*?Born:(\d*-\w*-\d*) \(Debut.*?Height:(\d*).*?Weight:(\d*) kg')



In [7]:
c_games = soup.find_all('table', attrs={'style':'font: 12px Verdana;', 'width':"100%", 'border':"1"})

for i in range(0,1):
    tables = c_games[i].find_all('td')

    r = i // 9 + 1
    id = i % 9 + 1

    gameid = '2022R' + str(r) + '0' + str(id)
    year = 2022
    round = 'R' + str(r)

    print(gameid)
    print(year)
    print(round)



    for j in range(len(tables)):





        if j == 0:

            home_team = tables[j].text
            home_team_index = 'https://afltables.com/afl' + tables[j].find_all('a', href=True)[0]['href'][2:]

            print(home_team)
            print(home_team_index)

        if j == 2:

            home_team_score = int(tables[j].text)
            print(home_team_score)

        if j == 3:

            temp = re_date_time_att_venue.match(tables[j].text)
            date = temp.group(1)
            venue = temp.group(4)
            start_time = temp.group(2)
            attendance = temp.group(3)
            attendance = re.sub(r'\D', '', attendance)
            attendance = int(attendance)



            print(date)
            print(venue)
            print(start_time)
            print(attendance)

        if j == 4:

            away_team = tables[j].text
            away_team_index = 'https://afltables.com/afl' + tables[j].find_all('a', href=True)[0]['href'][2:]

            print(away_team)
            print(away_team_index)

        if j == 6:

            away_team_score = int(tables[j].text)
            print(away_team_score)

        if j == 7:

            stats_link = 'https://afltables.com/afl' + tables[j].find_all('a', href=True)[0]['href'][2:]

            print(stats_link)

        print('#########')

    rainfall = 0
    print(rainfall)
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')


2022R101
2022
R1
Melbourne
https://afltables.com/afl/teams/melbourne_idx.html
#########
#########
97
#########
16-Mar-2022
M.C.G.
7:10 PM
58002
#########
Western Bulldogs
https://afltables.com/afl/teams/bullldogs_idx.html
#########
#########
71
#########
https://afltables.com/afl/stats/games/2022/071120220316.html
#########
0
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


In [8]:
# m = re.match(r'\w* (\d*\-\w*\-\d*) (\d*\:\d*\s\w*) \w*\: (\d*\,\d*) \w*\: (.*)', 'Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')

m = re.match(r'\w* (\d*\-\w*\-\d*) (\d*\:\d*\s\w*) .*?Att\: (\d*\,\d*) \w*\: (.*)', 'Sun 20-Mar-2022 4:40 PM (6:40 PM) Att: 20,932 Venue: Perth Stadium')


# m = re_date_time_att_venue.match('Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')

print(m)
print(m.group(1))
print(m.group(2))


n = m.group(3)
n = re.sub(r'\D', '', n)



print(n)
print(m.group(4))

<re.Match object; span=(0, 66), match='Sun 20-Mar-2022 4:40 PM (6:40 PM) Att: 20,932 Ven>
20-Mar-2022
4:40 PM
20932
Perth Stadium


In [150]:
p_data = new_players_data

# Crawling Website
url= 'https://afltables.com/afl/stats/games/2022/071120220316.html'


stats_datas = requests.get(url=url, headers=headers)

html_stats = stats_datas.text

# For Loop crawling, time interval is demanded.
sleep(0.25)

soup_stats = BeautifulSoup(html_stats, 'lxml')

In [151]:
c_stats_tables = soup_stats.find_all('table', attrs={'class' : "sortable"})
for i_table in range(0,2):
    # each table start here

    if i_table == 0:
        player_team = home_team
    if i_table == 1:
        player_team = away_team

    print(player_team)

    table_body = c_stats_tables[i_table].find_all('tbody')
    table_rows = table_body[0].find_all('tr')

    for table_row in table_rows:
        # each row start here

        table_columns = table_row.find_all('td')
        for p in range(len(table_columns)):
            # each column start here
            infos = table_columns[p].text
            if p != 0 and p != 1:
                if infos == '\xa0':
                    infos = 0

                infos = int(infos)

            if p == 0:
                infos =  re.sub(r'\D', '', infos)
                player_number = int(infos)
                subs = '-'

                table_players_rows = c_stats_tables[i_table+2].find_all('tbody')[0].find_all('tr')
                for table_players_row in table_players_rows:
                    table_players_columns = table_players_row.find_all('td')
                    if int(table_players_columns[p].text) == player_number:
                        gameNumber = re.match(r'(\d*) \(', table_players_columns[3].text)[1]
                        gameNumber = int(gameNumber)
                        print(gameNumber)

            if p == 1:
                displayName = infos
                print(displayName)

                name_url = 'https://afltables.com/afl/stats' + table_columns[p].find_all('a', href=True)[0]['href'][5:]
                team_player_info = query_player_info(name_url)
                player_dob = team_player_info[0]
                player_height = team_player_info[1]
                player_weight = team_player_info[2]

                selected_p = p_data[(p_data['displayName'] == displayName) & (p_data['dob'] == player_dob)]

                if selected_p.shape[0] == 0:
                    selected_p.at[0, 'playerId'] = int(8888866666)
                    selected_p.at[0, 'displayName'] = displayName
                    selected_p.at[0, 'height'] = player_height
                    selected_p.at[0, 'weight'] = player_weight
                    selected_p.at[0, 'dob'] = player_dob
                    selected_p.at[0, 'position'] = 'To_be_insert_manually'
                    selected_p.at[0, 'origin'] = 'To_be_insert_manually'
                    p_data = pd.concat([selected_p, p_data])

                playerId = selected_p['playerId'].iloc[0]

                print(playerId)



            if p == 2:
                Kicks = infos
                print(Kicks)
            if p == 3:
                Marks = infos
                print(Marks)
            if p == 4:
                Handballs = infos
                print(Handballs)
            if p == 5:
                Disposals = infos
                print(Disposals)
            if p == 6:
                Goals = infos
                print(Goals)
            if p == 7:
                Behinds = infos
                print(Behinds)
            if p == 8:
                Hit_Outs = infos
                print(Hit_Outs)
            if p == 9:
                Tackles = infos
                print(Tackles)
            if p == 10:
                Rebounds = infos
                print(Rebounds)
            if p == 11:
                Inside_50s = infos
                print(Inside_50s)
            if p == 12:
                Clearances = infos
                print(Clearances)
            if p == 13:
                Clangers = infos
                print(Clangers)
            if p == 14:
                Frees = infos
                print(Frees)
            if p == 15:
                Frees_Against = infos
                print(Frees_Against)
            if p == 16:
                Brownlow_Votes = infos
                print(Brownlow_Votes)
            if p == 17:
                Contested_Possessions = infos
                print(Contested_Possessions)
            if p == 18:
                Uncontested_Possessions = infos
                print(Uncontested_Possessions)
            if p == 19:
                Contested_Marks = infos
                print(Contested_Marks)
            if p == 20:
                Marks_Inside_50 = infos
                print(Marks_Inside_50)
            if p == 21:
                One_Percenters = infos
                print(One_Percenters)
            if p == 22:
                Bounces = infos
                print(Bounces)
            if p == 23:
                Goal_Assists = infos
                print(Goal_Assists)
            if p == 24:
                p_Played = infos
                print(p_Played)



        print('##########################')






Melbourne
3
Bedford, Toby
2020667638
5
3
4
9
0
1
0
2
1
1
0
1
2
0
0
4
5
0
1
2
1
0
55
##########################
8
Bowey, Jake
2021666343
8
2
1
9
1
0
0
0
2
1
1
1
0
0
0
3
6
0
0
2
0
0
73
##########################
120
Brayshaw, Angus
2015665345
12
6
11
23
0
0
0
3
2
2
0
4
1
1
0
4
19
0
0
1
0
0
83
##########################
144
Brown, Ben
2014665536
9
8
4
13
3
3
0
0
0
1
0
1
0
1
0
6
7
2
6
2
0
0
86
##########################
86
Fritsch, Bayley
2018705341
8
4
1
9
2
2
0
1
0
2
0
2
0
1
0
3
5
1
3
0
0
1
81
##########################
160
Gawn, Max
2011718646
11
4
3
14
1
0
34
1
2
5
4
9
2
6
0
12
3
2
0
3
0
0
97
##########################
123
Harmes, James
2015722143
6
1
11
17
1
0
0
5
1
3
2
4
2
1
0
7
10
1
1
3
0
0
77
##########################
95
Hunt, Jayden
2016725603
7
4
4
11
0
0
0
1
2
0
0
2
0
0
0
4
7
1
0
2
0
0
86
##########################
31
Jackson, Luke
2020747780
3
5
9
12
1
1
10
4
1
1
1
1
0
1
0
8
5
2
1
1
0
0
86
##########################
26
Jordon, James
2021745011
10
4
10
20
0
0
0
2
1
0
0
2
3
0
0


In [152]:
display(p_data)

,playerId,displayName,height,weight,dob,position,origin
0,2021661124,"Berry, Sam",181,81,12-Feb-2002,"Midfield, Forward",Gippsland Power
1,2012662083,"Brown, Luke",181,81,22-Sep-1992,Defender,Norwood
2,2020665315,"Butts, Jordon",198,83,31-Dec-1999,Defender,Murray Bushrangers
3,2021678665,"Cook, Brayden",189,80,18-Jul-2002,"Midfield, Forward",South Adelaide
4,2019683521,"Davis, Ben",187,85,19-May-1997,Forward,UNSW
5,2018689604,"Doedee, Tom",188,88,1-Mar-1997,Defender,Geelong Falcons
6,2018704381,"Fogarty, Darcy",193,94,25-Sep-1999,Forward,Glenelg
7,2018703883,"Frampton, Billy",200,90,20-Nov-1996,Forward,South Fremantle
8,2021712671,"Gollant, Lachlan",191,72,12-Sep-2001,"Defender, Midfield",Calder Cannons
9,2020728148,"Hamill, Will",186,71,17-Nov-2000,Defender,Dandenong Stingrays


0


,playerId,displayName,height,weight,dob,position,origin
0,2012888666.00,"Zhao, Wang",180.00,65.00,30-Mar-1994,FF,Sydney


           playerId                   displayName  height  weight  \
0     2012888666.00                    Zhao, Wang  180.00   65.00   
0     2021661124.00                    Berry, Sam  181.00   81.00   
1     2012662083.00                   Brown, Luke  181.00   81.00   
2     2020665315.00                 Butts, Jordon  198.00   83.00   
3     2021678665.00                 Cook, Brayden  189.00   80.00   
4     2019683521.00                    Davis, Ben  187.00   85.00   
5     2018689604.00                   Doedee, Tom  188.00   88.00   
6     2018704381.00                Fogarty, Darcy  193.00   94.00   
7     2018703883.00               Frampton, Billy  200.00   90.00   
8     2021712671.00              Gollant, Lachlan  191.00   72.00   
9     2020728148.00                  Hamill, Will  186.00   71.00   
10    2019721038.00               Hately, Jackson  190.00   83.00   
11    2018722651.00           Himmelberg, Elliott  198.00   94.00   
12    2019722465.00               

In [148]:
players.info(True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   playerId     1495 non-null   int64 
 1   displayName  1495 non-null   object
 2   height       1495 non-null   int64 
 3   weight       1495 non-null   int64 
 4   dob          1495 non-null   object
 5   position     1495 non-null   object
 6   origin       1491 non-null   object
dtypes: int64(3), object(4)
memory usage: 81.9+ KB


In [105]:
url = 'https://afltables.com/afl/stats/players/T/Toby_Bedford.html'

query_player_info(url)

['27-May-2000', 178, 69]